<a href="https://colab.research.google.com/github/Infinitysurya/AppliedMachineLearning/blob/main/AppliedML_Assignment5_Part2_EadaSuryaDeva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers datasets accelerate -q
!pip install -U tensorboard -q
# %%
from google.colab import drive
drive.mount('/content/drive')
# %%
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report
import numpy as np
# %%
# ---------------------------
# 1. Load and Preprocess Data
# ---------------------------

# Paths to your dataset files
train_path = "/content/drive/MyDrive/SentimentAnalysisData/SentimentAnalysisTrain.csv"
test_path = "/content/drive/MyDrive/SentimentAnalysisData/SentimentAnalysisTest.csv"

# Load CSVs (note: test file has no header)

# Load the dataset
train_df = pd.read_csv(train_path, encoding='latin-1')

# Take 10% of the data randomly and overwrite train_df
train_df = train_df.sample(frac=0.1, random_state=42)

# Optionally, reset index if needed
train_df.reset_index(drop=True, inplace=True)

# Now train_df contains only 10% of the original data

test_df = pd.read_csv(test_path, encoding='latin-1', header=None)

# Define column names based on the dataset description
columns = ['polarity', 'id', 'date', 'query', 'user', 'text']
train_df.columns = columns
test_df.columns = columns

# Keep only the polarity and text columns
train_df = train_df[['text', 'polarity']]
test_df = test_df[['text', 'polarity']]

# Filter valid polarity values (0, 2, 4)
train_df = train_df[train_df['polarity'].isin([0, 2, 4])]
test_df = test_df[test_df['polarity'].isin([0, 2, 4])]

# Map polarity to sentiment labels
sentiment_map = {0: 'Negative', 2: 'Neutral', 4: 'Positive'}
train_df['sentiment'] = train_df['polarity'].map(sentiment_map)
test_df['sentiment'] = test_df['polarity'].map(sentiment_map)

# Encode sentiment labels using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(['Negative', 'Neutral', 'Positive'])
train_df['label'] = label_encoder.transform(train_df['sentiment'])
test_df['label'] = label_encoder.transform(test_df['sentiment'])

# Prepare lists of texts and labels
train_texts = train_df['text'].tolist()
train_labels = train_df['label'].tolist()
test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()
# %%
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# ---------------------------
# 2. Tokenize the Data (Optimized with Parallel Processing)
# ---------------------------
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_batch(texts):
    return tokenizer.batch_encode_plus(
        texts,
        truncation=True,
        padding='max_length',  # Pad to max length
        max_length=128,
        return_tensors="pt"
    )


# Using ThreadPoolExecutor to process tokenization in parallel
def parallel_tokenize(texts, batch_size=64, num_workers=16):
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
        results = list(tqdm(executor.map(tokenize_batch, batches), total=len(batches)))
    # Combine the results from all batches
    input_ids = torch.cat([result['input_ids'] for result in results], dim=0)
    attention_mask = torch.cat([result['attention_mask'] for result in results], dim=0)
    return {'input_ids': input_ids, 'attention_mask': attention_mask}

# Tokenize train and test datasets in parallel
train_encodings = parallel_tokenize(train_texts, batch_size=128, num_workers=64)
test_encodings = parallel_tokenize(test_texts, batch_size=128, num_workers=64)



# ---------------------------
# 3. Create a Custom Dataset
# ---------------------------
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Return a dictionary with input_ids, attention_mask, and labels
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

# ---------------------------
# 4. Load the Pretrained Model
# ---------------------------

from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# Move model to GPU if available
if torch.cuda.is_available():
    model.to('cuda')

# ---------------------------
# 5. Set Up Training Arguments with GPU Support
# ---------------------------
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader, Subset
import numpy as np

# Reduce batch size and number of workers (adjust based on GPU memory)
train_loader = DataLoader(train_dataset, batch_size=32, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=8, pin_memory=True)

# Training arguments for GPU
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,  # Reduced batch size for GPU
    per_device_eval_batch_size=16,  # Reduced batch size for evaluation
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=4,  # Increased gradient accumulation
    report_to="none",
    disable_tqdm=False,
    dataloader_num_workers=4,  # Reduced number of workers
    logging_dir='./logs',
    fp16=torch.cuda.is_available(),

)

# ---------------------------
# Define the Trainer and Train the Model with 10% data
# ---------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start training
trainer.train()

# ---------------------------
# 7. Evaluate the Model and Output a Classification Report
# ---------------------------
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)

print(classification_report(test_labels, y_pred, target_names=label_encoder.classes_))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current 

Epoch,Training Loss,Validation Loss
0,0.323800,2.596177


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


              precision    recall  f1-score   support

    Negative       0.60      0.95      0.73       178
     Neutral       0.00      0.00      0.00       140
    Positive       0.57      0.67      0.61       198

    accuracy                           0.58       516
   macro avg       0.39      0.54      0.45       516
weighted avg       0.42      0.58      0.49       516



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
